In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install -q transformers datasets sentencepiece

!pip uninstall -y -q transformers accelerate
!pip install -q transformers accelerate

!pip install -q rouge-score

!pip install -q torchmetrics

!pip install -q git+https://github.com/huggingface/accelerate

!git clone -q https://github.com/huggingface/transformers.git
!cd transformers
!pip install  -q -e.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 84.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 16.5 MB/s eta 0:0

In [ ]:
# model_name_or_path = '/content/drive/MyDrive/pquad-mt5-base/checkpoint-16432'
# model_name_or_path = '/content/drive/MyDrive/mt5-pquad/checkpoint-4032'
# model_name_or_path ='/content/drive/MyDrive/mt5-pquad/checkpoint-470'
model_name_or_path ='/content/drive/MyDrive/mt5-pquad/checkpoint-3790'
from transformers import AutoTokenizer, MT5ForConditionalGeneration
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = MT5ForConditionalGeneration.from_pretrained(model_name_or_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from datasets import concatenate_datasets, load_dataset

data_files = {"train": "/content/drive/MyDrive/train.json", "validation": "/content/drive/MyDrive/dev.json"}
dataset = load_dataset("json", data_files=data_files)

train_data = dataset['train']
valid_data = dataset['validation']

concat_dataset = concatenate_datasets([train_data, valid_data])
dataset=concat_dataset

In [ ]:
dataset

Dataset({
    features: ['pq_id', 'answers', 'context', 'question'],
    num_rows: 2088
})

In [ ]:
del concat_dataset, train_data, valid_data

small_dataset = dataset

# small_dataset= dataset.filter(lambda example: len(example['answers'][0]['text'])>0)
# small_dataset= small_dataset.filter(lambda example: len(example['context'])<600)
# small_dataset= small_dataset.filter(lambda example: 'چرا' not in example['context'])

split_data = small_dataset.train_test_split(test_size=0.1, seed= 42, shuffle=True)

dataset = split_data

del split_data

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['pq_id', 'answers', 'context', 'question'],
        num_rows: 1879
    })
    test: Dataset({
        features: ['pq_id', 'answers', 'context', 'question'],
        num_rows: 209
    })
})

In [ ]:
def format_dataset(example):
  return {'input': 'answer: ' + example['answers'][0]['text'] + ' context: ' + example['context'], 'target': example['question']}


dataset = dataset.map(format_dataset, remove_columns=dataset['train'].column_names)

In [ ]:
test_data = dataset['test']

test_data[0]

{'input': 'answer: هر کس به دعوت کننده الهی پاسخ نگوید context:  هنگامی که گروهی از جنّ را به سوی تو متوجّه ساختیم که قرآن را بشنوند؛ وقتی حضور یافتند به یکدیگر گفتند: «خاموش باشید و بشنوید!» و هنگامی که پایان گرفت، به سوی قوم خود بازگشتند و آنها را بیم دادند! گفتند: «ای قوم ما! ما کتابی را شنیدیم که بعد از موسی نازل شده، هماهنگ با نشانه\u200cهای کتابهای پیش از آن، که به سوی حقّ و راه راست هدایت می\u200cکند. ای قوم ما! دعوت کننده الهی را اجابت کنید و به او ایمان آورید تا گناهانتان را ببخشد و شما را از عذابی دردناک پناه دهد! و هر کس به دعوت کننده الهی پاسخ نگوید، هرگز نمی\u200cتواند از چنگال عذاب الهی در زمین فرار کند، و غیر از خدا یار و یاوری برای او نیست؛ چنین کسانی در گمراهی آشکارند!»',
 'target': 'چه کسی هرگز نمی\u200cتواند از چنگال عذاب الهی در زمین فرار کند؟'}

In [ ]:
def convert_to_features(example_batch):
    input_encodings = tokenizer.batch_encode_plus(example_batch['input'], truncation=True, padding='max_length', max_length=564)
    target_encodings = tokenizer.batch_encode_plus(example_batch['target'], truncation=True, padding='max_length', max_length=64)

    encodings = {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids'],
        'decoder_attention_mask': target_encodings['attention_mask']
    }

    return encodings


In [ ]:
dataset = dataset.map(convert_to_features, batched=True, remove_columns=dataset['train'].column_names)

columns = ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask']

dataset.set_format(type='torch', columns=columns)

In [ ]:
# from transformers import default_data_collator
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq,Seq2SeqTrainingArguments, Seq2SeqTrainer
# data_collator = default_data_collator
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [ ]:
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments

# set training arguments - Feel free to adapt it
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/mt5-pquad",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    evaluation_strategy="epoch",
    do_train=True,
    do_eval=True,
    save_strategy="epoch",
    save_steps=8000,
    #eval_steps=1000,
    overwrite_output_dir=True,
    save_total_limit=3,
    load_best_model_at_end=True,
    # push_to_hub=False
    #fp16=True,
)

In [ ]:
# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    # data_collator=T2TDataCollator(),
    data_collator=data_collator,
    tokenizer=tokenizer,
    # compute_metrics=compute_metrics
)

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,0.565944


TrainOutput(global_step=470, training_loss=0.19717499753262135, metrics={'train_runtime': 222.0012, 'train_samples_per_second': 8.464, 'train_steps_per_second': 2.117, 'total_flos': 1094425112125440.0, 'train_loss': 0.19717499753262135, 'epoch': 1.0})

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

prompt = "answer: طور context: اي بني اسرائيل ما شما را از (چنگال) دشمنتان نجات داديم، و در طرف راست كوه طور با شما وعده گذارديم، و من و سلوي بر شما نازل كرديم.د"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generated_ids = model.generate(input_ids,
                                    num_beams=7,
                                    num_return_sequences=7,
#                                     top_k=10,
#                                     top_p=.2,
#                                     temperature=0.95,

                                    max_length=200)
for generated in generated_ids:
    generated_text = tokenizer.decode(generated,skip_special_tokens=True)
print(generated_text)

بنی اسرائيل در طرف راست چه طرفی با بنی اسرائيل وعده گذارند؟
